In [ ]:
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.stats import chisquare, chi2_contingency, ttest_ind, contingency, pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

import importlib
import sys

sys.path.append('/mnt/shared_folders/eResearch_glaucoma_project/andrewholmes2024/Aug2024')
import data_functions
importlib.reload(data_functions)

import feature_sets
importlib.reload(feature_sets)

In [ ]:
!jupyter nbconvert --to script ./data_extraction.ipynb

# Systemic diagnosis codes

In [ ]:
# Disease diagnosis dictionary
# Used to load in the required self-report & occurrence fields, and also for defining these diseases later

# All codes doubled-checked for accuracy 12 Sep 2024

diagnosis_dict = {
    ### Neurological & psychiatric
    'Dementia': {
        'self_reported': {
            'f.20002.0': [1263],
        },
        'occurrences': [
            'f.131036.0.0', 'f.130836.0.0', 
            'f.130838.0.0', 'f.130840.0.0', 'f.130842.0.0'
        ],
    },
    "Alzheimer's disease": {
        'occurrences': ['f.130836.0.0', 'f.131036.0.0'],
    },
     "Non-Alzheimer's dementia": {
        'occurrences': ['f.130838.0.0', 'f.130840.0.0', 'f.130842.0.0'],
    },
    'Migraine': {
        'self_reported': { 
            'f.20002.0': [1265],
        },
        'occurrences': ['f.131052.0.0'],
    },
    'Anxiety disorder': {
        'self_reported': {'f.20002.0': [1287]},
        'occurrences': ['f.130904.0.0', 'f.130906.0.0'], # F40, F41
    },
    'Conductive and sensorineural hearing loss': {
        'occurrences': ['f.131258.0.0'], # H90
    },
    'Other hearing loss': {
        'occurrences': ['f.131260.0.0'], # H91
    },

    
    ### Vascular
    'Hypertension': {
        'self_reported': {
            'f.20002.0': [1065, 1072],
            'f.6150.0': [4],
            'f.6177.0': [2],
            'f.6153.0': [2],
        },
        'occurrences': ['f.131286.0.0', 'f.131288.0.0', 'f.131290.0.0', 'f.131292.0.0', 'f.131294.0.0'],
    },
    'Hypotension': {
        'occurrences': ['f.131416.0.0'],
    },
    'Peripheral vascular disease': {
        'self_reported': {
            'f.20002.0': [1067, 1087, 1561],
        },
        'occurrences': ['f.131386.0.0'],
    },
    'Diabetes mellitus': {
        'self_reported': {
            'f.20002.0': [1220, 1222, 1223],
            'f.2443.0': [1],
            'f.6177.0': [3],
            'f.6153.0': [3],
        },
        'occurrences': ['f.130706.0.0', 'f.130708.0.0', 'f.130714.0.0'],
    },
    'Dyslipidaemia': {
        'self_reported': {
            'f.20002.0': [1473],
            'f.6177.0': [1],
            'f.6153.0': [1],
        },
        'occurrences': ['f.130814.0.0'],
    },

    
    ### Other cardio-metaoblic related
    'Sleep apnoea': {
        'self_reported': {
            'f.20002.0': [1123],
        },
        'inpatient': {
            'icd10': ['G473'],
            'icd9': ['3272'],
        },
        'gp': {
            'mapped_codes_dir': '/mnt/shared_folders/eResearch_glaucoma_project/andrewholmes2024/Aug2024/data/coding/primary_care/mapped_sleep_apnoea_primary_care.csv',
        },
    },
    'Atrial fibrillation or flutter': {
        'self_reported': {
            'f.20002.0': [1471, 1483], # 1471=afib, 1483 = aflutter
        },
        'occurrences': ['f.131350.0.0'],
    },

    
    ### Renal
    'Chronic kidney disease': {
        'self_reported': {
            'f.20002.0': [1192, 1193, 1194],
        },
        'occurrences': ['f.132032.0.0'],
    },


    ### Endocrine
    'Hypothyroidism': {
        'self_reported': {
            'f.20002.0': [1226],
        },
        'occurrences': ['f.130694.0.0', 'f.130696.0.0'],
    },
    'Thyrotoxicosis': {
        'self_reported': {
            'f.20002.0': [1522, 1225],
        },
        'occurrences': ['f.130700.0.0'],
    },

    
    ### Gastrointestinal
    'Gingivitis or periodontitis': {
        'occurrences': ['f.131562.0.0'],
    },
    'Helicobacter pylori infection': { 
        'self_reported': {
            'f.20002.0': [1442],
        },
        'inpatient': {
            'icd10': ['B980'],
            'icd9': [],
        },
        'gp': {
            'mapped_codes_dir': '/mnt/shared_folders/eResearch_glaucoma_project/andrewholmes2024/Aug2024/data/coding/primary_care/mapped_h_pylori_primary_care.csv',
        },
    },
    'Irritable bowel syndrome': { 
        'self_reported': {
            'f.20002.0': [1154],
        },
        'occurrences': ['f.131638.0.0'],
    },
 

    ### Auto-immune
    'Psoriasis': { 
        'self_reported': {
            'f.20002.0': [1453],
        },
        'occurrences': ['f.131742.0.0'],
    },
    'Sjogren syndrome': { 
        'self_reported': {
            'f.20002.0': [1382],
        },
        'inpatient': {
            'icd10': ['M350'],
            'icd9': ['7102'],
        },
        'gp': {
            'mapped_codes_dir': '/mnt/shared_folders/eResearch_glaucoma_project/andrewholmes2024/Aug2024/data/coding/primary_care/mapped_sjogren_syndrome_primary_care.csv',
        },
    },
    'Rheumatoid arthritis': { 
        'self_reported': { 
            'f.20002.0': [1464],
        },
        'occurrences': ['f.131848.0.0', 'f.131850.0.0'],
    },
    'Ankylosing spondylitis': { 
        'self_reported': {
            'f.20002.0': [1313]
        },
        'occurrences': ['f.131912.0.0'],
    },

    ### Other immune-related / inflammatory / atopic
    'Rosacea': { 
        'self_reported': {
            'f.20002.0': [1660],
        },
        'occurrences': ['f.131792.0.0'],
    },
     'COPD': { 
        'self_reported': {
            'f.20002.0': [1112, 1113, 1472],
            'f.6152.0': [6],
        },
        'occurrences': ['f.131486.0.0', 'f.131488.0.0', 'f.131490.0.0', 'f.131492.0.0'],
    },
    'Asthma': {
        'self_reported': {
            'f.20002.0': [1111],
            'f.6152.0': [8],
        },
        'occurrences': ['f.131494.0.0', 'f.131496.0.0'],
    },
    'Atopic dermatitis': { 
        'self_reported': {
            'f.20002.0': [1452],
        },
        'occurrences': ['f.131720.0.0'],
    },    
    'Vasomotor or allergic rhinitis': { 
        'self_reported': {
            'f.20002.0': [1387],
        },
        'occurrences': ['f.131464.0.0'],
    },
    'Chronic sinusitis': { 
        'self_reported': {
            'f.20002.0': [1416],
        },
        'occurrences': ['f.131468.0.0'],
    },
}

# Loading data

In [ ]:
def add_cols_for_data_field(data_field, data_cols, cols_to_use_list):
    cols_for_code = data_cols[data_cols.str.startswith(data_field)]
    if len(cols_for_code) < 1:
            print('No columns found matching field ' + data_field)
    cols_to_use_list.extend(cols_for_code)
    return cols_for_code

### (Load from previous work = can skip below)

- Loading from Pickle file is much faster than reading csv
- If loading from Pickle, can skip loading CSVs below, unless need to reimport original data

In [ ]:
# Load saved, RAW

# Valid 20 Sep 2024 all features
merged_df = pd.read_pickle('./data/raw_data_merged.pkl') 

In [ ]:
# Load saved, PROCESSED

# Valid 20 Sep 2024 all features
merged_df = pd.read_pickle('./data/derived/mixed_derived_and_extracted_merged.pkl') 

In [ ]:
# Load saved, derived only

# Valid 20 Sep 2024 all features
merged_df = pd.read_pickle('./data/derived/derived_cols_merged.pkl') 

In [ ]:
# GP events

# Valid 16 Sep 2024
merged_gp_events_df = pd.read_pickle('./data/extracted_gp_events_dates_merged.pkl')

### ukb49508

In [ ]:
# ukb49508 = assesement centre (NOT ocular), inpatient data

ukb49508_csv_cols = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/UKBB_Data/ukb49508.tab', nrows=0).columns
ukb49508_cols_to_use = [
    # Demographic
    'f.eid', 
    'f.34.0.0', # Year of birth
    'f.52.0.0', # Month of brith
    'f.53.0.0', # date attending assesement centre
    'f.21003.0.0', # age at assesement centre
] 

ukb49508_fields_to_add = [
    
    ### Demographic
    ####################################################

    'f.31.0.0', # Sex
    'f.21000.0.0', # Ethnicity
    'f.20118.0.0', # urban / rural
    'f.24006.0.0', # PM2.5

    'f.738.0.0', # Average household income
    'f.6138.0', # Qualifications (array)
    'f.4674.0.0', # Private healthcare use

    
    ### Systemic
    ####################################################

    # Hearing
    'f.2247.0.0', # Hearing difficulties (self-report)
    'f.4803.0.0', # Tinnitus (self-report)
    'f.4814.0.0', # Tinnitus severity (self-report)
    'f.20019.0.0', # Speech-reception-threshold, left
    'f.20021.0.0', # Speech-reception-threshold, right

    # Anthropometry
    'f.21001.0.0', # BMI
    'f.48.0.0', # Waist circumference
    'f.49.0.0', # Hip circumference

    # Blood pressure
    'f.4080.0.0', 'f.4080.0.1', # SBP automated
    'f.4079.0.0', 'f.4079.0.1', # DBP automated
    'f.93.0.0', 'f.93.0.1', # SBP manual
    'f.94.0.0', 'f.94.0.1', # DBP manual

    # Arterial stiffness
    'f.21021.0.0', # Pulse wave arterial stiffness index

    # Lipids
    'f.30870.0.0', # Triglycerides
    'f.30690.0.0', # Cholesterol
    'f.30760.0.0', # HDL
    'f.30780.0.0', # LDL direct
    'f.30630.0.0', # Apop A
    'f.30640.0.0', # Apop B

    # Endocrine: sex hormones
    'f.30800.0.0', # Oestradiol
    'f.30806.0.0', # Oestradiol reportability
    'f.30805.0.0', # Oestradiol missing reason ---> use 30806 instead (more detail)
    'f.30850.0.0', # Testosterone
    'f.30856.0.0', # Testosterone reportability
    'f.30855.0.0', # Testosterone missing reason ---> use 30856 instead

    # Blood counts
    'f.30080.0.0', # Platelet count
    'f.30120.0.0', # Lymphocyte count
    'f.30140.0.0', # Neutrophil count
    'f.30130.0.0', # Monocyte count
    
    # Other bloods
    'f.30750.0.0', # HBA1c
    'f.30740.0.0', # Glucose
    'f.30700.0.0', # Serum creatinine
    'f.30720.0.0', # Serum cystatin C
    'f.30880.0.0', # Serum urate
    'f.30840.0.0', # Total bilirubin
    'f.30710.0.0', # CRP
    'f.30600.0.0', # Albumin
    'f.30890.0.0', # Blood Vitamin D
    
    # Urine
    'f.30500.0.0', # Urine microalbumin
    'f.30505.0.0', # Urine microalbumin flag
    'f.30510.0.0', # Urine creatinine

    # Other
    'f.6149.0', # Oral health

    
    ### Lifestyle
    ####################################################

    # Social isolation
    'f.2020.0.0', # feelings of loneliness
    'f.2110.0.0', # how often confide in others
    'f.1031.0.0', # frequency of friend/famiyl visits
    'f.6160.0', # leisure/social activities; array
    'f.709.0.0', # number in household
    
    # Exercise
    'f.22037.0.0', # MET mins/week walking
    'f.22038.0.0', # MET mins/week moderate
    'f.22039.0.0', # MET mins/week vigorous
    #'f.22040.0.0', # Summed MET ---> not available
    
    'f.864.0.0', # days/week walking
	'f.874.0.0', # mins walking per day
    'f.884.0.0', # days/week moderate activity
	'f.894.0.0', # duration moderate activity p/d
    'f.904.0.0', # days/week vigorous activity
	'f.914.0.0', # duration vigorus activity p/d

    'f.6164.0', # types of activity in last 4 weeks (array)
    'f.1001.0.0', # duration of strenuous sports

	# Sleep
	'f.1160.0.0', # sleep duration
	'f.1200.0.0', # insomnia
	'f.1210.0.0', # snoring
	'f.1220.0.0', # daytime somnolence

	# Smoking
    'f.1239.0.0', # current tobacco smoking
    'f.1249.0.0', # past tobacco smoking (not collected for those who indicate they currently smoke most/all days in 1239)

    'f.3456.0.0', # current cigarretes daily (if most/all days in 1239)
    'f.2887.0.0', # previous cigarretes daily (if most/all days in 1249)
	'f.20161.0.0', # pack years ever smoked

	# Alcohol
	'f.1558.0.0', # current alcohol intake frequency

	# Coffee & tea
	# 'f.1498.0.0', # coffee intake cups/d -> NOT AVAILABLE
	'f.1508.0.0', # coffee type (is NaN if no intake)
	'f.1488.0.0', # tea intake cups/d

    # Healthy diet score, from touchscreen (Lourida et al., 2019)
    'f.1309.0.0', # Fresh fruit intake
    'f.1319.0.0', # Dried fruit intake
    'f.1289.0.0', # Cooked vegetable intake
    'f.1299.0.0', # Salad/raw vegetable intake
    'f.1329.0.0', # Oily fish intake
    'f.1339.0.0', # Non-oily fish intake
    'f.1349.0.0', # Processed meat intake
    'f.1369.0.0', # Beef intake
    'f.1379.0.0', # Lamb/mutton intake
    'f.1389.0.0', # Pork intake
    'f.1438.0.0', # Bread intake
    'f.1448.0.0', # Bread type
    'f.1458.0.0', # Cereal intake
    'f.1468.0.0', # Cereal type

	# Other dietary markers
    'f.1478.0.0', # Salt added to food
	'f.30530.0.0', # Urinary Na+ excretion (to calculate UNa:Cr ratio)

    # Supplements
    'f.6155.0', # Vitamin supp, array -> Vitamin C
    'f.6179.0', # Mineral/other supp, array -> calcium, iron, selenium
    
    # QC
    'f.1538.0.0', # Major dietary changes in last 5 years (see 2: 'because of illness')
    'f.1548.0.0', # Dietary variation week-to-week
]


for data_field in ukb49508_fields_to_add:
    add_cols_for_data_field(data_field, ukb49508_csv_cols, ukb49508_cols_to_use)

occurrence_date_cols = []
for disease, vals in diagnosis_dict.items():
    if 'self_reported' in vals:
        for data_field, v in vals['self_reported'].items():
            add_cols_for_data_field(data_field, ukb49508_csv_cols, ukb49508_cols_to_use)

# Medication
self_report_medication_cols = add_cols_for_data_field('f.20003.0', ukb49508_csv_cols, ukb49508_cols_to_use) # self-reported medication codes

# Inpatient
ICD10_inpatient_dx_cols  = add_cols_for_data_field('f.41270.0', ukb49508_csv_cols, ukb49508_cols_to_use) # ICD10 inpatient diagnosis 
ICD10_inpatient_date_cols = add_cols_for_data_field('f.41280.0', ukb49508_csv_cols, ukb49508_cols_to_use) # ICD10 inpatient date

ICD9_inpatient_dx_cols = add_cols_for_data_field('f.41271.0', ukb49508_csv_cols, ukb49508_cols_to_use) # ICD9 inpatient diagnosis 
ICD9_inpatient_date_cols = add_cols_for_data_field('f.41281.0', ukb49508_csv_cols, ukb49508_cols_to_use) # ICD9 inpatient date

ukb49508_cols_to_use = np.unique(ukb49508_cols_to_use)
print(f'Found {len(ukb49508_cols_to_use)} columns')

In [ ]:
# Read ukb49508

ukb49508_df = pd.read_table(
    '/mnt/shared_folders/eResearch_glaucoma_project/UKBB_Data/ukb49508.tab', 
    usecols=ukb49508_cols_to_use, 
    low_memory=True,
    parse_dates = np.concatenate((['f.53.0.0'], ICD10_inpatient_date_cols, ICD9_inpatient_date_cols)).tolist(),
    date_format='%Y-%m-%d',
)

In [ ]:
# Save a raw copy

ukb49508_df.to_pickle('./data/extracted_raw_ukb49508.pkl', compression=None)

In [ ]:
# Load raw copy

ukb49508_df = pd.read_pickle('./data/extracted_raw_ukb49508.pkl')

### ukb675501

In [ ]:
# ukb675501 = deprivation index, occurrences

ukb675501_csv_cols = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/UKBB_Data_healthrelatedoutcomes/ukb675501.tab', nrows=0).columns
ukb675501_cols_to_use = [
    'f.eid',
    'f.22189.0.0', # Townsend Deprivation Index
]

occurence_date_cols = []
for disease, vals in diagnosis_dict.items():
    if 'occurrences' in vals:
        for data_field in vals['occurrences']:
             occurence_date_cols.extend(add_cols_for_data_field(data_field, ukb675501_csv_cols, ukb675501_cols_to_use))

In [ ]:
# Read ukb675501

ukb675501_df = pd.read_table(
    '/mnt/shared_folders/eResearch_glaucoma_project/UKBB_Data_healthrelatedoutcomes/ukb675501.tab', 
    usecols=ukb675501_cols_to_use, 
    low_memory=True,
    parse_dates = occurence_date_cols,
    date_format='%Y-%m-%d',
)

In [ ]:
# Save a raw copy

ukb675501_df.to_pickle('./data/extracted_raw_ukb675501.pkl', compression=None)

In [ ]:
# Load raw copy

ukb675501_df = pd.read_pickle('./data/extracted_raw_ukb675501.pkl')

### ukb51745

In [ ]:
# ukb51745 = ocular data

ukb51745_csv_cols = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/UKBB_Data_Basket3/ukb51745.tab', nrows=0).columns
ukb51745_cols_to_use = ['f.eid']

ukb51745_fields_to_add = [
    # Self-report
    ##########################

    # Self-reported myopia
    'f.2207.0', # Wears glasses or lenses
    'f.6147.0', # Reason for glasses / lenses
    'f.5843.0', # Which eye(s) affected by myopia
    
    # Tonometry
    ##########################

    # IOPcc
    'f.5262.0', #'f.5262.1', # OS (left eye)
    'f.5254.0', #'f.5254.1', # OD (right eye)

    # IOPg
    'f.5263.0', # OS
    'f.5255.0', # OD

    # Corneal hysteresis
    'f.5264.0', #'f.5264.1', # OS
    'f.5256.0', #'f.5256.1', # OD

    # Corneal resistance factor
    'f.5265.0', #'f.5265.1', # OS
    'f.5257.0', #'f.5257.1', # OS


    # Refractometry
    # Using refractometer 1 (=end of recuritment & 2012 repeat visit)
    # (not used: Refractometer 2 = 2022 re-imaging visit)
    ##########################

    # Rfractometry result unreliable (QC)
    'f.5090.0', #'f.5090.1', # OS
    'f.5091.0', #'f.5091.1', # OD

    # Best index
    'f.5276.0', #'f.5276.1', # OS
    'f.5221.0', #'f.5221.1', # OD

    # Cylindrical power
    'f.5086.0', #'f.5086.1', # OS
    'f.5087.0', #'f.5087.1', # OD

    # Spherical power
    'f.5085.0', #'f.5085.1', # OS
    'f.5084.0', #'f.5084.1', # OD

    #  VA logMAR final
    'f.5208.0.0', #'f.5208.1.0', # OS
    'f.5201.0.0', #'f.5201.1.0', # OD


    # OCT derived measurements
    ##########################

    # Average RNFL thickness
    'f.28500.0', #'f.28500.1', # OS
    'f.28501.0', #'f.28501.1', # OD

    # Average GC-IPL thickness
    'f.28504.0', #'f.28504.1', # OS
    'f.28505.0', #'f.28505.1', # OD

    # Overall macular thickness
    'f.27800.0', #'f.27800.1', # OS
    'f.27801.0', #'f.27801.1', # OD

    # vCDR
    'f.27857.0', #'f.27857.1', # OS
    'f.27858.0', #'f.27858.1', # OD ----> very few participants? (OS 'more reliable' as measured 2nd)

    # Mean of vertical disc diameter
    'f.27853.0', #'f.27853.1', # OS
    'f.27854.0', #'f.27854.1', # OD  ----> very few participants? (OS 'more reliable' as measured 2nd)

    # QC, image quality
    # 'f.28552.0', 'f.28552.1', # OS
    # 'f.28553.0', 'f.28553.1', # OD
]


for data_field in ukb51745_fields_to_add:
    add_cols_for_data_field(data_field, ukb51745_csv_cols, ukb51745_cols_to_use)

In [ ]:
# Read ukb51745

ukb51745_df = pd.read_table(
    '/mnt/shared_folders/eResearch_glaucoma_project/UKBB_Data_Basket3/ukb51745.tab',
    usecols=ukb51745_cols_to_use, 
    low_memory=True,
    date_format='%Y-%m-%d',
)

In [ ]:
# Save a raw copy

ukb51745_df.to_pickle('./data/extracted_raw_ukb51745.pkl', compression=None)

In [ ]:
# Load raw copy

ukb51745_df = pd.read_pickle('./data/extracted_raw_ukb51745.pkl')

### Research group csv for PRS

In [ ]:
prs_df = pd.read_table(
    '/mnt/shared_folders/eResearch_glaucoma_project/UKBB_Data_mtDNA/paper_yonovadoing_etal/variableinput_apply_PRS_to_everything.txt',
    usecols=['Participant_ID', 'Standardized_PRS_Craig2020'],
    low_memory=True, 
    date_format='%Y-%m-%d'
).rename(columns={'Participant_ID':'f.eid', 'Standardized_PRS_Craig2020': 'Polygenic risk score'})

### Merge data files

In [ ]:
# Merge data files

merged_df = pd.merge(ukb49508_df, ukb675501_df, on='f.eid', how='outer')
merged_df = pd.merge(merged_df, ukb51745_df, on='f.eid', how='outer')
merged_df = pd.merge(merged_df, prs_df, on='f.eid', how='outer')

#merged_df = ukb51745_df
#merged_df = ukb49508_df

In [ ]:
# Save merged data file

merged_df.to_pickle('./data/raw_data_merged.pkl') 

### GP data

In [ ]:
# Load GP clinical events

gp_events_raw_table = pd.read_table(
    '/mnt/shared_folders/eResearch_glaucoma_project/UKBB_Data_healthrelatedoutcomes/gp_clinical.txt', 
    low_memory=True, 
    usecols=['eid', 'event_dt', 'read_2', 'read_3'],
    parse_dates = ['event_dt'],
    date_format='%d/%m/%Y',
)

merged_gp_events_df = gp_events_raw_table.merge(merged_df[['f.eid', 'f.53.0.0']], left_on='eid', right_on='f.eid', suffixes=('_gp', '_inpatient'))

In [ ]:
# Save GP files

gp_events_raw_table.to_pickle('./data/extracted_raw_gp_events.pkl')
merged_gp_events_df.to_pickle('./data/extracted_gp_events_dates_merged.pkl')

# Ophthalmic data processing

### Intraocular pressure (IOP)

**IOPcc**

In [ ]:
# IOPcc

merged_df['IOPcc_OS'] = merged_df['f.5262.0.0']
merged_df['IOPcc_OD'] = merged_df['f.5254.0.0']

# Get constant eye - worse eye for IOP
merged_df['IOPcc'] = merged_df[['IOPcc_OS', 'IOPcc_OD']].max(axis=1, skipna=True)

# Define IOP subcohort
IOP_available_mask = merged_df['IOPcc'].isna() == False
merged_df['IOP_available'] = IOP_available_mask.astype(int)

### ! Swapped to using IOPg
# Define worse eye, for future use
# If there is a tie (n=156), use OS (left eye measured 2nd, so potentially more reliable)
# OS_worse_IOPcc_mask = (merged_df['IOPcc'] == merged_df['IOPcc_OS']) & IOP_available_mask
# OD_worse_IOPcc_mask = (merged_df['IOPcc'] == merged_df['IOPcc_OD']) & IOP_available_mask & (~OS_worse_IOPcc_mask)
# merged_df.loc[OS_worse_IOPcc_mask, 'constant_eye'] = 'OS'
# merged_df.loc[OD_worse_IOPcc_mask, 'constant_eye'] = 'OD'

In [ ]:
# IOPcc asymmetry
# Will be NaN if other eye missing

merged_df['IOPcc inter-eye difference'] = merged_df['IOPcc'] - (merged_df[['IOPcc_OS', 'IOPcc_OD']].min(axis=1, skipna=False))

**Pre-treatment IOPcc imputation**

In [ ]:
# Get IOP medication usage at baseline assesement visit (i=0)
# From self-report only (ignore GP)

iop_medications = pd.read_csv('./data/coding/IOP_medications.csv')
iop_medication_codes = iop_medications['UKBB_Code']

merged_df['on_IOP_medication_baseline'] = merged_df[self_report_medication_cols].isin(iop_medication_codes.values).any(axis=1).astype(int)
on_IOP_medication_mask = merged_df['on_IOP_medication_baseline'] == 1

In [ ]:
# Impute pre-treatment IOPcc

treatment_IOP_multiplier = 1.3

# IOPcc OS
merged_df['IOPcc_OS_pretreatment'] = merged_df['IOPcc_OS']
merged_df.loc[on_IOP_medication_mask, 'IOPcc_OS_pretreatment'] = merged_df[on_IOP_medication_mask]['IOPcc_OS'] * treatment_IOP_multiplier

# IOPcc OD
merged_df['IOPcc_OD_pretreatment'] = merged_df['IOPcc_OD']
merged_df.loc[on_IOP_medication_mask, 'IOPcc_OD_pretreatment'] = merged_df[on_IOP_medication_mask]['IOPcc_OD'] * treatment_IOP_multiplier

# IOPcc constant eye (eye with highest IOP)
merged_df['IOPcc pre-treatment'] = merged_df['IOPcc']
merged_df.loc[on_IOP_medication_mask, 'IOPcc pre-treatment'] = merged_df[on_IOP_medication_mask]['IOPcc'] * treatment_IOP_multiplier

**IOPg**

In [ ]:
# IOPg

merged_df['IOPg_OS'] = merged_df['f.5263.0.0']
merged_df['IOPg_OD'] = merged_df['f.5255.0.0']

# Get constant eye - worse eye for IOP
# Note: in ~20,000 people, this eye is different, compared to IOPcc
merged_df['IOPg'] = merged_df[['IOPg_OS', 'IOPg_OD']].max(axis=1, skipna=True)

# Define worse eye, for future use
# If there is a tie (n=156), use OS (left eye measured 2nd, so potentially more reliable)
OS_worse_IOPg_mask = (merged_df['IOPg_OS'] >= merged_df['IOPg_OD']) | ((merged_df['IOPg_OD'].isna() == True) & (merged_df['IOPg_OS'].isna() == False))
OD_worse_IOPg_mask = (merged_df['IOPg_OS'] < merged_df['IOPg_OD']) | ((merged_df['IOPg_OS'].isna() == True) & (merged_df['IOPg_OD'].isna() == False))

merged_df.loc[OS_worse_IOPg_mask, 'constant_eye'] = 'OS'
merged_df.loc[OD_worse_IOPg_mask, 'constant_eye'] = 'OD'

In [ ]:
# IOPg asymmetry
# Will be NaN if other eye missing

merged_df['IOPg inter-eye difference'] = (merged_df['IOPg_OS'] - merged_df['IOPg_OD']).abs()

**Pre-treatment IOPg imputation**

In [ ]:
# Impute pre-treatment IOPcc

treatment_IOP_multiplier = 1.3

# IOPg OS
merged_df['IOPg_OS_pretreatment'] = merged_df['IOPg_OS']
merged_df.loc[on_IOP_medication_mask, 'IOPg_OS_pretreatment'] = merged_df[on_IOP_medication_mask]['IOPg_OS'] * treatment_IOP_multiplier

# IOPg OD
merged_df['IOPg_OD_pretreatment'] = merged_df['IOPg_OD']
merged_df.loc[on_IOP_medication_mask, 'IOPg_OD_pretreatment'] = merged_df[on_IOP_medication_mask]['IOPg_OD'] * treatment_IOP_multiplier

# IOPg eye with higher IOP
merged_df['IOPg pre-treatment'] = merged_df['IOPg']
merged_df.loc[on_IOP_medication_mask, 'IOPg pre-treatment'] = merged_df[on_IOP_medication_mask]['IOPg'] * treatment_IOP_multiplier

# IOPg asymmetry
merged_df['IOPg pre-treatment inter-eye difference'] = (merged_df['IOPg_OS_pretreatment'] - merged_df['IOPg_OD_pretreatment']).abs()

### Corneal hysteresis (CH)

In [ ]:
# CH

merged_df['corneal_hysteresis_OS'] = merged_df['f.5264.0.0']
merged_df['corneal_hysteresis_OD'] = merged_df['f.5256.0.0']

# # In consistent eye (highest IOP), according to IOPcc
# merged_df.loc[OS_worse_IOPcc_mask, 'Corneal hysteresis'] = merged_df['corneal_hysteresis_OS'] 
# merged_df.loc[OD_worse_IOPcc_mask, 'Corneal hysteresis'] = merged_df['corneal_hysteresis_OD'] 

# In consistent eye (highest IOP), according to IOPg
merged_df.loc[OS_worse_IOPg_mask, 'Corneal hysteresis'] = merged_df['corneal_hysteresis_OS'] 
merged_df.loc[OD_worse_IOPg_mask, 'Corneal hysteresis'] = merged_df['corneal_hysteresis_OD'] 

In [ ]:
# CH asymmetry
# Will be NaN if other eye missing

# Using IOPg
# Abs

merged_df['Corneal hysteresis inter-eye difference'] = (merged_df['corneal_hysteresis_OS'] - merged_df['corneal_hysteresis_OD']).abs()

### Corneal resistance factor (CRF)

In [ ]:
# CRF

merged_df['corneal_resistance_factor_OS'] = merged_df['f.5265.0.0']
merged_df['corneal_resistance_factor_OD'] = merged_df['f.5257.0.0']

# # In consistent eye (highest IOP), according to IOPcc
# merged_df.loc[OS_worse_IOPcc_mask, 'Corneal resistance factor'] = merged_df['corneal_resistance_factor_OS'] 
# merged_df.loc[OD_worse_IOPcc_mask, 'Corneal resistance factor'] = merged_df['corneal_resistance_factor_OD'] 

# In consistent eye (highest IOP), according to IOPg
merged_df.loc[OS_worse_IOPg_mask, 'Corneal resistance factor'] = merged_df['corneal_resistance_factor_OS'] 
merged_df.loc[OD_worse_IOPg_mask, 'Corneal resistance factor'] = merged_df['corneal_resistance_factor_OD'] 

### Refractive error

**Self-reported: wears glasses for myopia**

In [ ]:
# Self-reported myopia in any eye
# Field 6147, data code 1
merged_df['self_reported_myopia_any_eye'] = merged_df[merged_df.columns[merged_df.columns.str.startswith('f.6147.0.')]].eq(1).any(axis=1).astype(int)

# Self-reported myopia in the constant eye (eye with highest IOP)
# Field 5843: 1 = OD, 2 = OS, 3 = both eyes
myopia_OS_mask = merged_df['f.5843.0.0'].eq(2)
myopia_OD_mask = merged_df['f.5843.0.0'].eq(1)
myopia_both_mask = merged_df['f.5843.0.0'].eq(3)

# # According to IOPcc
# merged_df['Self-reported myopia'] = 0
# merged_df.loc[OS_worse_IOPcc_mask & myopia_OS_mask, 'Self-reported myopia'] = 1
# merged_df.loc[OD_worse_IOPcc_mask & myopia_OD_mask, 'Self-reported myopia'] = 1
# merged_df.loc[myopia_both_mask, 'Self-reported myopia'] = 1

# merged_df['Self-reported myopia which eye'] = pd.NA
# merged_df.loc[myopia_OS_mask, 'Self-reported myopia which eye'] = 'OS'
# merged_df.loc[myopia_OD_mask, 'Self-reported myopia which eye'] = 'OD'
# merged_df.loc[myopia_both_mask, 'Self-reported myopia which eye'] = 'Both'

# According to IOPg
merged_df['Self-reported myopia'] = 0
merged_df.loc[OS_worse_IOPg_mask & myopia_OS_mask, 'Self-reported myopia'] = 1
merged_df.loc[OD_worse_IOPg_mask & myopia_OD_mask, 'Self-reported myopia'] = 1
merged_df.loc[myopia_both_mask, 'Self-reported myopia'] = 1

merged_df['Self-reported myopia which eye'] = pd.NA
merged_df.loc[myopia_OS_mask, 'Self-reported myopia which eye'] = 'OS'
merged_df.loc[myopia_OD_mask, 'Self-reported myopia which eye'] = 'OD'
merged_df.loc[myopia_both_mask, 'Self-reported myopia which eye'] = 'Both'

# Process NA, where we don't know if they wear glasses for myopia:
# NaN or -3 for 2207 (wears glasses)
# -1 or -3 for 6147 (reason for glasses)
unknown_glasses_for_myopia_mask = merged_df[merged_df.columns[merged_df.columns.str.startswith('f.2207.0.')]].eq(-3).any(axis=1) | merged_df[merged_df.columns[merged_df.columns.str.startswith('f.6147.0.')]].isin([-1, -3]).any(axis=1)
merged_df.loc[unknown_glasses_for_myopia_mask, 'self_reported_myopia_any_eye'] = np.nan # any eye
merged_df.loc[unknown_glasses_for_myopia_mask, 'Self-reported myopia'] = np.nan # constant eye

**Spherical equivalent via auto-refractor**

In [ ]:
merged_df['cylindrical_power_OS'] = merged_df[merged_df.columns[merged_df.columns.str.startswith('f.5086.0')]].mean(axis=1, skipna=True)
merged_df['cylindrical_power_OD'] = merged_df[merged_df.columns[merged_df.columns.str.startswith('f.5087.0')]].mean(axis=1, skipna=True)

merged_df['spherical_power_OS'] = merged_df[merged_df.columns[merged_df.columns.str.startswith('f.5085.0')]].mean(axis=1, skipna=True)
merged_df['spherical_power_OD'] = merged_df[merged_df.columns[merged_df.columns.str.startswith('f.5084.0')]].mean(axis=1, skipna=True)

# SE = S + C/2
merged_df['spherical_equivalent_OS'] = merged_df['spherical_power_OS'] + (0.5 * merged_df['cylindrical_power_OS'])
merged_df['spherical_equivalent_OD'] = merged_df['spherical_power_OD'] + (0.5 * merged_df['cylindrical_power_OD'])

In [ ]:
# # Spherical equivalent in consistent eye (highest IOP), according to IOPcc
# merged_df.loc[OS_worse_IOPcc_mask, 'Spherical equivalent'] = merged_df['spherical_equivalent_OS'] 
# merged_df.loc[OD_worse_IOPcc_mask, 'Spherical equivalent'] = merged_df['spherical_equivalent_OD'] 

# Spherical equivalent in consistent eye (highest IOP), according to IOPg
merged_df.loc[OS_worse_IOPg_mask, 'Spherical equivalent'] = merged_df['spherical_equivalent_OS'] 
merged_df.loc[OD_worse_IOPg_mask, 'Spherical equivalent'] = merged_df['spherical_equivalent_OD'] 

**Save derived fields**

In [ ]:
# Save refraction measurements (for future research group use)

merged_df[['f.eid', 'self_reported_myopia_any_eye', 'Self-reported myopia which eye', 'spherical_equivalent_OS', 'spherical_equivalent_OD']].to_csv('./data/derived/ukbb_refractive_error.tsv', sep='\t', index=False)

# Demographic data processing

### Simple columns: rename

In [ ]:
# Age
merged_df['Age at initial assesement'] = merged_df['f.21003.0.0']

# Sex
merged_df['Sex'] = merged_df['f.31.0.0'] # 0 = female, 1 = male

# PM2.5 exposure
merged_df['PM2.5 exposure'] = merged_df['f.24006.0.0']

### Ethnicity

- Feature for models: `white_ethnicity`, binary dichotomous. Code as White / Non-White. Non-White includes "Mixed" ethnicities
- For other descriptive statistics: `detailed_ethnicity`, as per `ethnicity_groupings` (recoded from UKB defaults)

In [ ]:
ethnicity_groupings = {
    'white': [1, 1001, 1002, 1003],
    'mixed': [2, 2001, 2002, 2003, 2004],
    'asian': [3, 5, 3001, 3002, 3003, 3004],
    'black': [4, 4001, 4002, 4003],
    'other': [6],
}

In [ ]:
# Detailed variable: `detailed_ethnicity`
# One-hot encoded as `ethnicity_`

for group_name, data_codes in ethnicity_groupings.items():
    ethnicity_members = merged_df['f.21000.0.0'].isin(data_codes)
    merged_df.loc[ethnicity_members, 'detailed_ethnicity'] = group_name

    merged_df[f'ethnicity_{group_name}'] = 0
    merged_df.loc[ethnicity_members, f'ethnicity_{group_name}'] = 1

In [ ]:
# Summary variable: `white_ethnicity`

merged_df['Ethnicity'] = merged_df['f.21000.0.0'].isin(ethnicity_groupings['white']).astype(int)

# Handle NaN
merged_df.loc[merged_df['detailed_ethnicity'].isna() == True, 'Ethnicity'] = np.nan

for group_name, data_codes in ethnicity_groupings.items():
        merged_df.loc[merged_df['detailed_ethnicity'].isna() == True, f'ethnicity_{group_name}'] = np.nan

### Urban/rural

- `is_urban`: Binary dichotomous variable
- Consistent urban definition across England/Wales & Scotland: >10,000 population area

In [ ]:
# Data codes corresponding to urban areas, i.e.
# England/Wales: 1 (Urban sparse), 5 (Urban less sparse)
# Scotland: 11 (Large Urban rea), 12 (Other Urban Area)
urban_data_codes = [1, 5, 11, 12]

# Set variable
merged_df['Urban residence'] = merged_df['f.20118.0.0'].isin(urban_data_codes).astype(int)

# Handle NaN
merged_df.loc[merged_df['f.20118.0.0'].isna() == True, 'Urban residence'] = np.nan

### Socioeconomic deprivation

In [ ]:
# Townsend Deprivation index (TDI)
merged_df['Townsend deprivation index'] = merged_df['f.22189.0.0']

In [ ]:
# Average household income before tax
# Ordinal, per default UKB codings

# 1	Less than 18,000
# 2	18,000 to 30,999
# 3	31,000 to 51,999
# 4	52,000 to 100,000
# 5	Greater than 100,000

# ! Large proportion missing (decline to answer)

merged_df['Total household income'] = merged_df['f.738.0.0'].replace({
    -1: np.nan,
    -3: np.nan,
})

In [ ]:
# Qualifications
# Ordinal

# 0 = none
# 1 = secondary education (A levels, O levels, CSEs)
# 2 = higher education (University degree,  NVQ/HND/HNC, other professional qualifications e.g. nursing, teaching)

# Defualt coding in UKB:
# 1	College or University degree
# 2	A levels/AS levels or equivalent
# 3	O levels/GCSEs or equivalent
# 4	CSEs or equivalent
# 5	NVQ or HND or HNC or equivalent
# 6	Other professional qualifications eg: nursing, teaching
# -7	None of the above
# -3	Prefer not to answer

education_cols = merged_df.columns[merged_df.columns.str.startswith('f.6138.0')]

no_education_listed = merged_df['f.6138.0.0'].eq(-7)
secondary_education = merged_df[education_cols].isin([2, 3, 4]).any(axis=1)
higher_education = merged_df[education_cols].isin([1, 5, 6]).any(axis=1)
education_na = (merged_df['f.6138.0.0'].eq(-3)) | (merged_df['f.6138.0.0'].isna() == True)

merged_df['Education'] = np.nan # reset
merged_df.loc[no_education_listed, 'Education'] = 0
merged_df.loc[secondary_education, 'Education'] = 1
merged_df.loc[higher_education, 'Education'] = 2
merged_df.loc[education_na, 'Education'] = np.nan

In [ ]:
# Private healthcare utilisation
# Ordinal; higher value = higher private utilisation
# Recoded from UKB

# ! Large missing %, but available for most people with IOP (? added towards end)

# Default UKB:
# 1	Yes, all of the time
# 2	Yes, most of the time
# 3	Yes, sometimes
# 4	No, never
# -1	Do not know
# -3	Prefer not to answer

merged_df['Private healthcare utilisation'] = merged_df['f.4674.0.0'].replace({
    -1: np.nan,
    -3: np.nan,
    1: 3, # 1	Yes, all of the time
    2: 2, # 2	Yes, most of the time
    3: 1, # 3	Yes, sometimes
    4: 0, # 4	No, never
})

# Systemic data processing

## Diagnoses

In [ ]:
%%capture cap --no-stderr

diagnosis_case_breakdown_df = pd.DataFrame(columns=['disease', 'N baseline (% of cases)', 'Baseline, % of UKB', 'N baseline occurrence (%)', 'N baseline self-report (%)', 'N baseline inpatient (%)', 'N baseline GP (%)'])

for disease, vals in diagnosis_dict.items():
    # Initialise
    baseline_col_str = disease + ' (baseline)'
    incident_col_str = disease + ' (incident)'
    merged_df[baseline_col_str] = 0
    merged_df[incident_col_str] = 0
    
    print(f'\n{disease} \n{baseline_col_str} \n{incident_col_str}')
    print('------------------------------------------------')
    
    ### Handle occurrences (baseline & incident)
    baseline_occurrence_feids = set()
    incident_occurrence_feids = set()

    if 'occurrences' in vals:
        print('Processing occurrences')
        feid_dates_df = pd.DataFrame(columns=['f.eid', 'earliest_occurrence_diagnosis_date'])
        for occurrence_field in vals['occurrences']:
            n_baseline_invalid, n_incident_invalid, feid_dates_df = data_functions.extract_occurrence(merged_df, occurrence_field, baseline_occurrence_feids, incident_occurrence_feids, feid_dates_df)
            print(f'Invalid N for occurrence {occurrence_field}: baseline: {n_baseline_invalid} incident: {n_incident_invalid}')
        print(f'Earliest occurrence diagnosis dates: \n{feid_dates_df.groupby('f.eid')['earliest_occurrence_diagnosis_date'].min().describe()}')

    
    ### Handle self-report (all baseline)
    
    self_report_feids = set()
    if 'self_reported' in vals:
        print('Processing self-report')
        for data_field, positive_data_codes in vals['self_reported'].items():
            cols = merged_df.columns[merged_df.columns.str.startswith(data_field)]
            positive_cases = merged_df[cols].isin(positive_data_codes).any(axis=1)
            self_report_feids.update(merged_df[positive_cases]['f.eid'])


    
    ### Handle inpatient (baseline & incident)
    
    inpatient_diagnosis_dates = pd.DataFrame(columns=['f.eid', 'earliest_inpatient_diagnosis_date'])
    baseline_inpatient_feids = set()
    incident_inpatient_feids = set()
    
    if 'inpatient' in vals:
        earliest_date_inpatient = data_functions.extract_inpatient(
            merged_df,
            vals['inpatient']['icd10'],
            vals['inpatient']['icd9'],
            baseline_inpatient_feids,
            incident_inpatient_feids,
            inpatient_diagnosis_dates,
        )
        print(f'Earliest inpatient diagnosis dates: \n{earliest_date_inpatient.describe()}')


    ### Handle GP (baseline & incident)
    
    baseline_gp_feids = set()
    incident_gp_feids = set()

    if 'gp' in vals:
        # Read in coding
        coding_dir = vals['gp']['mapped_codes_dir']
        coding_df = pd.read_csv(coding_dir)
        read_v2_codes = coding_df[coding_df['read_category'] == 'v2']['read_code'].values
        read_v3_codes = coding_df[coding_df['read_category'] == 'v3']['read_code'].values
        print(f'Read v2 codes used: {', '.join(read_v2_codes)}')
        print(f'Read v3 codes used: {', '.join(read_v3_codes)}')

        earliest_date_gp, n_baseline_invalid, n_incident_invalid = data_functions.extract_gp(merged_gp_events_df, read_v2_codes, read_v3_codes, baseline_gp_feids, incident_gp_feids)
        print(f'Invalid N for GP: baseline: {n_baseline_invalid} incident: {n_incident_invalid}')
        print(f'Earliest GP diagnosis dates: \n{earliest_date_gp.describe()}')

    
    ### Generate masks
    
    baseline_diagnosis_from_occurrence = merged_df['f.eid'].isin(baseline_occurrence_feids)
    incident_diagnosis_from_occurrence = merged_df['f.eid'].isin(incident_occurrence_feids)

    baseline_diagnosis_from_self_report = merged_df['f.eid'].isin(self_report_feids)

    baseline_diagnosis_from_inpatient = merged_df['f.eid'].isin(baseline_inpatient_feids) 
    incident_diagnosis_from_inpatient = merged_df['f.eid'].isin(incident_inpatient_feids)

    baseline_diagnosis_from_gp = merged_df['f.eid'].isin(baseline_gp_feids)
    incident_diagnosis_from_gp = merged_df['f.eid'].isin(incident_gp_feids)

    all_baseline = baseline_diagnosis_from_occurrence | baseline_diagnosis_from_self_report | baseline_diagnosis_from_inpatient | baseline_diagnosis_from_gp
    all_incident = incident_diagnosis_from_occurrence | incident_diagnosis_from_inpatient | incident_diagnosis_from_gp
    all_cases = all_baseline | all_incident

    print(f'Mask sum N (may overlap):'
        + f'\nSelf report: {baseline_diagnosis_from_self_report.sum()}'
        + f'\nOccurrences, baseline: {baseline_diagnosis_from_occurrence.sum()}'
        + f'\nOccurrences, incident: {incident_diagnosis_from_occurrence.sum()}'
        + f'\nInpatient, baseline: {baseline_diagnosis_from_inpatient.sum()}'
        + f'\nInpatient, incident: {incident_diagnosis_from_inpatient.sum()}'
        + f'\nGP, baseline: {baseline_diagnosis_from_gp.sum()}'
        + f'\nGP, incident: {incident_diagnosis_from_gp.sum()}'
    )

    print(f'Case N:'
        + f'\nAll: {all_cases.sum()}'
        + f'\nBaseline: {all_baseline.sum()}'
        + f'\nIncident: {all_incident.sum()}'
   )

    #### Set diagnoses
    merged_df.loc[all_baseline, baseline_col_str] = 1
    merged_df.loc[all_incident, incident_col_str] = 1


    ### Update displays

    # Get "additional cases" at baseline identified in the following order: occurrences -> self-report -> inpatient -> GP
    # (Occurrences first, as this theoretically includes self-report)
    baseline_self_report_additional = baseline_diagnosis_from_self_report & ~baseline_diagnosis_from_occurrence
    baseline_inpatient_additional = baseline_diagnosis_from_inpatient & ~baseline_diagnosis_from_occurrence & ~baseline_diagnosis_from_self_report
    baseline_gp_additional = baseline_diagnosis_from_gp & ~baseline_diagnosis_from_occurrence & ~baseline_diagnosis_from_self_report & ~baseline_diagnosis_from_inpatient

    diagnosis_case_breakdown_df.loc[len(diagnosis_case_breakdown_df)] = [
        disease, # disease
        f'{all_baseline.sum()} ({((all_baseline.sum() / all_cases.sum()) * 100):0.2f}%)', # N baseline (%)
        f'{((all_baseline.sum() / len(merged_df)) * 100):0.2f}%', # Baseline % of UKB
        f'{baseline_diagnosis_from_occurrence.sum()} ({((baseline_diagnosis_from_occurrence.sum() / all_baseline.sum()) * 100):0.2f}%)', # N occurrence_B (%)
        f'{baseline_self_report_additional.sum()} ({((baseline_self_report_additional.sum() / all_baseline.sum()) * 100):0.2f}%)', # N self-report_B (%)
        f'{baseline_inpatient_additional.sum()} ({((baseline_inpatient_additional.sum() / all_baseline.sum()) * 100):0.2f}%)', # N inpatient (%)
        f'{baseline_gp_additional.sum()} ({((baseline_gp_additional.sum() / all_baseline.sum()) * 100):0.2f}%)', # N GP-B (%)
    ]

    print(merged_df[baseline_col_str].value_counts())
    print(merged_df[incident_col_str].value_counts())

In [ ]:
# Save above output

with open('./data/qc/systemic_diagnoses_log.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
# Save derived baseline/incident columns for future use

save_cols = ['f.eid']

for disease, vals in diagnosis_dict.items():
    baseline_col_str = disease + ' (baseline)'
    incident_col_str = disease + ' (incident)'
    save_cols.append(baseline_col_str)
    save_cols.append(incident_col_str)

merged_df[save_cols].to_csv('./data/derived/ukbb_systemic_diagnoses.tsv', sep='\t', index=False)

## Other systemic factors

### Medications

In [ ]:
drugs_list = pd.read_table('./data/coding/export_systemic_drugs.tsv')
ukbb_drug_coding = pd.read_table('./data/coding/ukbb_meds_coding4.tsv')

In [ ]:
drug_classes_to_feature_names = {
    # drug class: feature name
    #'bupropion': 'Bupropion',
    'metformin': 'Metformin',
    'statin': 'Statin',
    'beta_blocker_systemic': 'Beta blocker',
    'CCB': 'Calcium channel blocker',
    'ACE_inhibitor': 'ACE inhibitor',
    'ARB': 'Angiotensin receptor blocker',
    'diuretic': 'Diuretic',
    'SSRI': 'SSRI',
    'SNRI': 'SNRI',
}

data_code_values = drugs_list['data_code'].values

In [ ]:
# Code medications
# Each are binary variables

for drug_class, feature_name in drug_classes_to_feature_names.items():
    drug_class_data_codes = drugs_list[(drugs_list[['class_1', 'class_2', 'class_3']] == drug_class).any(axis=1)]['data_code']
    merged_df[feature_name] = merged_df[self_report_medication_cols].isin(drug_class_data_codes.values).any(axis=1).astype(int)

In [ ]:
# Number of uses on each drug

drug_user_number_df = pd.DataFrame(columns=['data_code', 'med_name', 'N individuals'])

for i, row in drugs_list.iterrows():
    n_users = merged_df[self_report_medication_cols].eq(row['data_code']).any(axis=1).sum()
    drug_user_number_df.loc[i] = [row['data_code'], row['med_name'], n_users]

In [ ]:
drug_user_number_df = drug_user_number_df.sort_values(axis=0, by='N individuals', inplace=False, ascending=False)
drug_user_number_df.to_csv('./data/qc/systemic_drugs_n_users.tsv',  index=False, sep='\t')

In [ ]:
# Save derived medication coding for future use

save_cols = ['f.eid'] + list(drug_classes_to_feature_names.values())
merged_df[save_cols].to_csv('./data/derived/ukbb_systemic_medication_use.tsv', index=False, sep='\t')

**Recode drug classes for publication**

In [ ]:
recoded_df = pd.DataFrame(columns=['Data code', 'Medication name', 'Feature(s)'])
feature_names = drug_classes_to_feature_names.values()

for i, row in drugs_list.iterrows():
    #print(row['med_name'])
    drug_classes = row[['class_1', 'class_2', 'class_3']].astype(str).values
    #print(classes)
    #features = np.intersect1d(classes, drug_classes)
    features = [] #np.array([])
    for drug_class in drug_classes:
        if drug_class in drug_classes_to_feature_names:
            feature_name = drug_classes_to_feature_names[drug_class]
            features.append(feature_name)

    if len(features) == 0:
        continue

    # Removed duplicates
    resorted_features = [i for n, i in enumerate(features) if i not in features[:n]]
    
    recoded_df.loc[len(recoded_df)] = [row['data_code'], row['med_name'], ', '.join(resorted_features)]

In [ ]:
recoded_df = recoded_df.sort_values(axis=0, by=['Feature(s)', 'Medication name'], inplace=False, ascending=False)

In [ ]:
recoded_df.to_csv('./data/coding/systemic_drugs_publication_table.csv', index=False)

### Hearing

- Also see self-report diagnosis fields

**Self-reported hearing difficulties**

In [ ]:
# Hearing difficulty
# Binary var

# f. 2247
# Treat 99 (completely deaf) as yes

merged_df['Hearing difficulty (self-reported)'] = merged_df['f.2247.0.0'].replace({
    -1: np.nan,
    -3: np.nan,
    99: 1,
    1: 1,
    0: 0,
})

In [ ]:
# Tinnitus
# Ordinal variable - frequency of tinnitus - higher value = more frequent
# f. 4803

# ! Large proportion missing (only ~ n=172,000  available)
# However, available for most people with IOP (? added towards end of visits)

# Recoded
# Default UKB codings:
# 11	Yes, now most or all of the time
# 12	Yes, now a lot of the time
# 13	Yes, now some of the time
# 14	Yes, but not now, but have in the past
# 0	No, never
# -1	Do not know
# -3	Prefer not to answer

# n.b. code "past" as "No" (0)

merged_df['Tinnitus frequency (self-reported)'] = merged_df['f.4803.0.0'].replace({
    -1: np.nan,
    -3: np.nan,

    # Not currently
    0: 0, # never
    14: 0, # Yes, but not now, but have in the past

    # Yes
    13: 1, # Yes, now some of the time
    12: 2, # Yes, now a lot of the time
    11: 3, 	# Yes, now most or all of the time
})

**Hearing test**
- Limited N., but overlaps with IOP substudy
- Use better ear, as per past studies

In [ ]:
# Speech reception threshold (SRT)

merged_df['speech_reception_threshold_left'] = merged_df['f.20019.0.0']
merged_df['speech_reception_threshold_right'] = merged_df['f.20021.0.0']

# Best ear -> main variable of interest
# Defined as the ear with the lower SRT (lower signal:noise ratio i.e. less signal)
# If only one ear available, take that as the best ear
merged_df['Speech reception threshold'] = merged_df[['speech_reception_threshold_left', 'speech_reception_threshold_right']].min(axis=1, skipna=True)

### Blood pressure & arterial stiffness

In [ ]:
# Systolic BP
# Average all available automated and manual readings

sbp_cols = ['f.4080.0.0', 'f.4080.0.1', 'f.93.0.0', 'f.93.0.1']
merged_df['Systolic blood pressure'] = merged_df[sbp_cols].mean(axis=1, skipna=True)


# Diastolic BP
# Average all available automated and manual readings

dbp_cols = ['f.4079.0.0', 'f.4079.0.1', 'f.94.0.0', 'f.94.0.1']
merged_df['Diastolic blood pressure'] = merged_df[dbp_cols].mean(axis=1, skipna=True)

In [ ]:
# Arterial stiffness index
# Added towards end of assesement visit; available for most people with IOP measurement

merged_df['Arterial stiffness index'] = merged_df['f.21021.0.0']

### Self-reported oral health

- Using definition from Lee et al., 2024 (https://pubmed.ncbi.nlm.nih.gov/38506820/)

In [ ]:
# Poor oral health, self-reported
# Binary variable


# UKB f 6149 default coding:
# 1	Mouth ulcers
# 2	Painful gums
# 3	Bleeding gums
# 4	Loose teeth
# 5	Toothache
# 6	Dentures
# -7	None of the above
# -3	Prefer not to answer

# Poor oral health defined as any of: painful gums, bleeding gums, loose teeth, toothache, or dentures
# i.e. all responses except mouth ulcers (1)
# See Lee et al., 2024 (https://pubmed.ncbi.nlm.nih.gov/38506820/)
poor_oral_health_codes = [2, 3, 4, 5, 6]

cols_6149 = merged_df.columns[merged_df.columns.str.startswith('f.6149.0')]
merged_df['Poor oral health'] = merged_df[cols_6149].isin(poor_oral_health_codes).any(axis=1).astype(int)

# Preserve NA
f_6149_na = (merged_df['f.6149.0.0'].eq(-3)) | (merged_df['f.6149.0.0'].isna() == True)
merged_df.loc[f_6149_na, 'Poor oral health'] = np.nan

### Endocrine: sex hormones

- Large proportion of oestradiol & testosterone(to a lesser extent) is NaN due to being undetectable: see 'missing reason' & 'reportability' data fields

Reportable ranges from UKBB (https://biobank.ndph.ox.ac.uk/showcase/ukb/docs/serum_biochemistry.pdf):
- Oestradiol: <175 pmol/L
- Testosterone: <0.35 pmol/L
- Note: these are DIFFERENT to the assay analytical range (73-17621 pmol/L for oestradiol, 0.35-55.52 for testosterone)

We replace missing values (specifically due to being too low) with half the lower detectable limit (https://pubmed.ncbi.nlm.nih.gov/33156012/)

In [ ]:
### Oestradiol

merged_df['Plasma oestradiol'] = merged_df['f.30800.0.0']

# Lower limit for reporting: 175 pmol/L
# Threshold for missing values: 87.5
oestradiol_reportability_too_low = merged_df['f.30806.0.0'].isin([2, 4])
merged_df.loc[oestradiol_reportability_too_low, 'Plasma oestradiol'] = 87.5

In [ ]:
### Testosterone

merged_df['Plasma testosterone'] = merged_df['f.30850.0.0']

# Lower limit for reporting: 0.35 pmol/L
# Threshold for missing values: 0.175
testosterone_reportability_too_low = merged_df['f.30856.0.0'].isin([2, 4])
merged_df.loc[testosterone_reportability_too_low, 'Plasma testosterone'] = 0.175

### Blood markers

**Diabetes mellitus markers / insulin resistance**

In [ ]:
merged_df['HbA1c'] = merged_df['f.30750.0.0']
merged_df['Plasma glucose'] = merged_df['f.30740.0.0']

**Lipids**

In [ ]:
merged_df['Triglycerides'] = merged_df['f.30870.0.0']
merged_df['Total cholesterol'] = merged_df['f.30690.0.0']
merged_df['HDL'] = merged_df['f.30760.0.0']
merged_df['LDL'] = merged_df['f.30780.0.0']
merged_df['Apolipoprotein A'] = merged_df['f.30630.0.0']
merged_df['Apolipoprotein B'] = merged_df['f.30640.0.0']

**Inflammatory cell ratios**

- Platelet:lymphocyte ratio (PLR)
- Neutrophil:lymphocyte ratio (NLR)
- Systemic immune-inflammation index (SII) -> incorporates platelets, neutrophils, lymphocytes

In [ ]:
merged_df['platelet_count'] = merged_df['f.30080.0.0']
merged_df['lymphocyte_count'] = merged_df['f.30120.0.0']
merged_df['neutrophil_count'] = merged_df['f.30140.0.0']

# Zero lymphocytes -> avoid, as otherwise will divide by zero
zero_lymphocyte_count = merged_df['lymphocyte_count'] == 0

# PLR
merged_df['platelet_lymphocyte_ratio'] = merged_df['platelet_count'] / merged_df['lymphocyte_count']
merged_df.loc[zero_lymphocyte_count, 'platelet_lymphocyte_ratio'] = np.nan

# NLR
merged_df['neutrophil_lymphocyte_ratio'] = merged_df['neutrophil_count'] / merged_df['lymphocyte_count']
merged_df.loc[zero_lymphocyte_count, 'neutrophil_lymphocyte_ratio'] = np.nan

# SII
merged_df['Systemic immune inflammation index'] = merged_df['platelet_count'] * merged_df['neutrophil_count'] / merged_df['lymphocyte_count']
merged_df.loc[zero_lymphocyte_count, 'Systemic immune inflammation index'] = np.nan

**Other blood markers**

In [ ]:
merged_df['Plasma urate'] = merged_df['f.30880.0.0']
merged_df['Plasma total bilirubin'] = merged_df['f.30840.0.0']
merged_df['C-reactive protein'] = merged_df['f.30710.0.0']
merged_df['Plasma albumin'] = merged_df['f.30600.0.0']

### CKD markers

**eGFR**

- CKD-EPI 2021 creatinine equation (no ethnicity) https://www.niddk.nih.gov/research-funding/research-programs/kidney-clinical-research-epidemiology/laboratory/glomerular-filtration-rate-equations/adults https://pubmed-ncbi-nlm-nih-gov.ezproxy.auckland.ac.nz/34554658/
- Creatinine (f 30700) comes as  umol/L, needs to be converted to mg/dL

In [ ]:
merged_df['serum_creatinine_mgdl'] = merged_df['f.30700.0.0'] * 0.0113

In [ ]:
# Male

is_male = merged_df[merged_df['Sex'] == 1]
s_cr = is_male['serum_creatinine_mgdl']
age = is_male['Age at initial assesement']

K = 0.9
a = -0.302

ones_arr = np.ones(len(is_male))
eGFR_male = 142 * np.minimum(s_cr / K, ones_arr) ** a * np.maximum(s_cr / K, ones_arr) ** -1.200 * 0.9938 ** age

In [ ]:
# Female

is_female = merged_df[merged_df['Sex'] == 0]
s_cr = is_female['serum_creatinine_mgdl']
age = is_female['Age at initial assesement']

K = 0.7
a = -0.241

ones_arr = np.ones(len(is_female))
eGFR_female = 142 * np.minimum(s_cr / K, ones_arr) ** a * np.maximum(s_cr / K, ones_arr) ** -1.200 * 0.9938 ** age * 1.012

In [ ]:
# Set

merged_df['eGFR serum creatinine'] = np.nan
merged_df.loc[is_male.index, 'eGFR serum creatinine'] = eGFR_male
merged_df.loc[is_female.index, 'eGFR serum creatinine'] = eGFR_female

**Urinary albmin:creatinine ratio**
- Unit: mg / mmol
- Albumin (30500) is mg/L
- CCreatinine (30510) is micromole/L


In [ ]:
# Unit conversion, creatinine from micromole/L to mmol/L

merged_df['urine_creatinine_mmol'] = merged_df['f.30510.0.0'] / 1000

In [ ]:
# Set patients with undetectable albumin <6.7 mg/L to 6.7 mg/L

undetectable_albumin_mask = merged_df['f.30505.0.0'] == '<6.7' # below 6.7 mg/L
merged_df['urine_albumin'] = merged_df['f.30500.0.0']
merged_df.loc[undetectable_albumin_mask, 'urine_albumin'] = 6.7

In [ ]:
# ACR, unit mg/mmol

merged_df['Albumin-creatinine ratio'] = merged_df['urine_albumin'] / merged_df['urine_creatinine_mmol']

# Lifestyle data processing

### Healthy diet score

Adapted from Lourida et al., 2019 (https://pubmed.ncbi.nlm.nih.gov/31302669/)

1 point per 7 areas of diet

In [ ]:
def code_numerical_intake_column(df, input_column, output_column):
    # Codes special value -10 (<1 serving/day) as 0.5
    # Codes responses of -3 or -1 as NA
    df[output_column] = df[input_column]
    low_val = df[input_column].eq(-10)  # <1 serving/day, treat as 0.5
    decline = df[input_column].isin([-1, -3])
    df.loc[low_val, output_column] = 0.5
    df.loc[decline, output_column] = np.nan

In [ ]:
### Fruit intake score
# To get the point: >= 3 servings/day

# Recode fresh fruit intake for invalid values
code_numerical_intake_column(merged_df, 'f.1309.0.0', 'fresh_fruit_intake')

# Recode dried fruit intake for invalid values
code_numerical_intake_column(merged_df, 'f.1319.0.0', 'dried_fruit_intake')

# Define score
merged_df['diet_score_fruit_component'] = (merged_df[['fresh_fruit_intake', 'dried_fruit_intake']].sum(skipna=False, axis=1) >= 3).astype(int)

# Preserve na values
merged_df.loc[merged_df['fresh_fruit_intake'].isna() == True, 'diet_score_fruit_component'] = np.nan
merged_df.loc[merged_df['dried_fruit_intake'].isna() == True, 'diet_score_fruit_component'] = np.nan

In [ ]:
### Vegetable intake score
# To get the point: >= 3 servings/day

# Recode cooked vege intake for invalid values
code_numerical_intake_column(merged_df, 'f.1289.0.0', 'cooked_vegetable_intake')

# Recode dried fruit intake for invalid values
code_numerical_intake_column(merged_df, 'f.1299.0.0', 'raw_vegetable_intake')

# Define score
merged_df['diet_score_vegetable_component'] = (merged_df[['cooked_vegetable_intake', 'raw_vegetable_intake']].sum(skipna=False, axis=1) >= 3).astype(int)

# Preserve na values
merged_df.loc[merged_df['cooked_vegetable_intake'].isna() == True, 'diet_score_vegetable_component'] = np.nan
merged_df.loc[merged_df['raw_vegetable_intake'].isna() == True, 'diet_score_vegetable_component'] = np.nan

In [ ]:
### Fish intake score
# To get the point: >=2 servings/week

# Coding for oily fish and non-oily fish individually:
# 0 if never
# 0.5 if <1 per week
# 1 if once per week
# 2 if 2-4 times a week or more

# Code oily fish
merged_df['oily_fish_intake_group'] = merged_df['f.1329.0.0'].replace({
    -1: np.nan, # do not know
    -3: np.nan, # prefer not to answer
    
    # 0: Never / <1 per week
    0: 0, # never

    # 0.5:  <1 per week
    1: 0.5, # <1 once per week

    # 1: Once a week
    2: 1,

    # 2: >= 2-4 times a week
    3: 2, # 2-4 a week
    4: 2, # 5-6 a week
    5: 2, # once or more daily
})

# Code non-oily fish
merged_df['non_oily_fish_intake_group'] = merged_df['f.1339.0.0'].replace({
    -1: np.nan, # do not know
    -3: np.nan, # prefer not to answer
    
    # 0: Never
    0: 0, # never

    # 0.5:  <1 per week
    1: 0.5, # <1 once per week

    # 1: Once a week
    2: 1,

    # 2: >= 2-4 times a week
    3: 2, # 2-4 a week
    4: 2, # 5-6 a week
    5: 2, # once or more daily
})


# Define score
merged_df['diet_score_fish_component'] = (merged_df[['oily_fish_intake_group', 'non_oily_fish_intake_group']].sum(skipna=False, axis=1) >= 2).astype(int)

# Preserve na values
merged_df.loc[merged_df['oily_fish_intake_group'].isna() == True, 'diet_score_fish_component'] = np.nan
merged_df.loc[merged_df['non_oily_fish_intake_group'].isna() == True, 'diet_score_fish_component'] = np.nan

In [ ]:
### Processed meat intake score
# To get the point: <=1 serving/week

# Define score
# (0 = never, 1 = less than once/week, 2 = once a week)
merged_df['diet_score_processed_meat_component'] = merged_df['f.1349.0.0'].isin([0, 1, 2]).astype(int)

# Preserve na values
merged_df.loc[merged_df['f.1349.0.0'].isna() == True, 'diet_score_processed_meat_component'] = np.nan
merged_df.loc[merged_df['f.1349.0.0'].isin([-1, -3]), 'diet_score_processed_meat_component'] = np.nan

In [ ]:
### Unprocessed red meat intake score
# To get the point: <=1.5 servings/week
# (beef, lamb/mutton, pork)

# Code "less than once a week" as 0.5 in this instance

# Coding for each meat intake type:
# 0 if never
# 0.5 if <1 once per week
# 1 if once per week
# 2 if 2-4 times a week or more

# Code beef intake
merged_df['beef_intake_group'] = merged_df['f.1369.0.0'].replace({
    -1: np.nan, # do not know
    -3: np.nan, # prefer not to answer
    
    # 0: Never
    0: 0, # never

    # 0.5: <1 once per week
    1: 0.5, # <1 once per week

    # 1: Once a week
    2: 1,

    # 2: >= 2-4 times a week
    3: 2, # 2-4 a week
    4: 2, # 5-6 a week
    5: 2, # once or more daily
})


# Code lamb/mutton intake
merged_df['lamb_intake_group'] = merged_df['f.1379.0.0'].replace({
    -1: np.nan, # do not know
    -3: np.nan, # prefer not to answer
    
    # 0: Never
    0: 0, # never

    # 0.5: <1 once per week
    1: 0.5, # <1 once per week

    # 1: Once a week
    2: 1,

    # 2: >= 2-4 times a week
    3: 2, # 2-4 a week
    4: 2, # 5-6 a week
    5: 2, # once or more daily
})

# Code pork intake
merged_df['pork_intake_group'] = merged_df['f.1389.0.0'].replace({
    -1: np.nan, # do not know
    -3: np.nan, # prefer not to answer
    
    # 0: Never
    0: 0, # never

    # 0.5: <1 once per week
    1: 0.5, # <1 once per week

    # 1: Once a week
    2: 1,

    # 2: >= 2-4 times a week
    3: 2, # 2-4 a week
    4: 2, # 5-6 a week
    5: 2, # once or more daily
})


# Define score
merged_df['diet_score_red_meat_component'] = (merged_df[['beef_intake_group', 'lamb_intake_group', 'pork_intake_group']].sum(skipna=False, axis=1) <= 1.5).astype(int)

# Preserve na values
merged_df.loc[merged_df['beef_intake_group'].isna() == True, 'diet_score_red_meat_component'] = np.nan
merged_df.loc[merged_df['lamb_intake_group'].isna() == True, 'diet_score_red_meat_component'] = np.nan
merged_df.loc[merged_df['pork_intake_group'].isna() == True, 'diet_score_red_meat_component'] = np.nan



In [ ]:
### Grains scores

# Set defaults
merged_df['whole_grains_servings_daily'] = np.nan
merged_df['refined_grains_servings_daily'] = np.nan

## Handle bread

# Bread intake
code_numerical_intake_column(merged_df, 'f.1438.0.0', 'bread_intake_weekly')

# Bread type
# Bread type not collected from those who specified no bread
bread_whole = merged_df['f.1448.0.0'].isin([3]) # Wholemeal
bread_refined = merged_df['f.1448.0.0'].isin([1, 2, 4]) # White, Brown, Other
bread_decline =  merged_df['f.1448.0.0'].isin([-1, -3])

# Whole grain servings from bread
merged_df.loc[bread_whole, 'whole_grains_servings_daily'] = merged_df.loc[bread_whole, 'bread_intake_weekly'] / 7

# Refined grain servings from bread
merged_df.loc[bread_refined, 'refined_grains_servings_daily'] = merged_df.loc[bread_refined, 'bread_intake_weekly'] / 7


## Handle cereals

# Cereal intake
code_numerical_intake_column(merged_df, 'f.1458.0.0', 'cereal_intake_weekly')
merged_df['cereal_intake_daily'] = merged_df['cereal_intake_weekly'] / 7

# Cereal type
# Cereal type not collected from those who specified no bread
cereal_whole = merged_df['f.1468.0.0'].isin([1, 2, 3, 4]) # Bran, biscuit, oat, muesli
cereal_refined = merged_df['f.1468.0.0'].eq(5) # Other (e.g. cornflakes)
cereal_decline =  merged_df['f.1468.0.0'].isin([-1, -3])

# Whole grain servings from cereal
merged_df.loc[cereal_whole, 'whole_grains_servings_daily'] = merged_df[cereal_whole][['whole_grains_servings_daily', 'cereal_intake_daily']].sum(axis=1, skipna=True)

# Refined grain servings from cereal
merged_df.loc[cereal_refined, 'refined_grains_servings_daily'] = merged_df[cereal_refined][['refined_grains_servings_daily', 'cereal_intake_daily']].sum(axis=1, skipna=True)


## Define scores

# Define whole grain score
# To get the point: >= 3 servings/day
merged_df['diet_score_whole_grains_component'] = (merged_df['whole_grains_servings_daily'] >= 3).astype(int)

# Define refined grain score
# To get the point: <= 1.5 servings/day
merged_df['diet_score_refined_grains_component'] = (merged_df['refined_grains_servings_daily'] <= 1.5).astype(int)

# Preserve na values
# Set both scores to NA if we are unsure of either their cereal or bread intake amount, or if declined to answer bread/cereal type
merged_df.loc[merged_df['bread_intake_weekly'].isna() == True, 'diet_score_whole_grains_component'] = np.nan
merged_df.loc[merged_df['bread_intake_weekly'].isna() == True, 'diet_score_refined_grains_component'] = np.nan
merged_df.loc[merged_df['cereal_intake_weekly'].isna() == True, 'diet_score_whole_grains_component'] = np.nan
merged_df.loc[merged_df['cereal_intake_weekly'].isna() == True, 'diet_score_refined_grains_component'] = np.nan
merged_df.loc[bread_decline | cereal_decline, 'diet_score_whole_grains_component'] = np.nan
merged_df.loc[bread_decline | cereal_decline, 'diet_score_refined_grains_component'] = np.nan

In [ ]:
### Combined diet score

merged_df['Diet score'] = merged_df[[
    'diet_score_fruit_component',
    'diet_score_vegetable_component',
    'diet_score_fish_component',
    'diet_score_processed_meat_component',
    'diet_score_red_meat_component',
    'diet_score_whole_grains_component',
    'diet_score_refined_grains_component'
]].sum(axis=1, skipna=False)

In [ ]:
# Save diet score (for future research group use)

merged_df[['f.eid', 'Diet score']].to_csv('./data/derived/ukbb_diet_score_Lourida_et_al_2019.tsv', sep='\t', index=False)

### Supplements

In [ ]:
# Vitamin C, multivtamin supplementation (f 6155)

f_6155_cols = merged_df.columns[merged_df.columns.str.startswith('f.6155.0.')]
f_6155_na_mask = (merged_df['f.6155.0.0'].eq(-3)) | (merged_df['f.6155.0.0'].isna() == True)

# Vitamin C supplementation
merged_df['Vitamin C supplementation'] = merged_df[f_6155_cols].eq(3).any(axis=1).astype(int)
merged_df.loc[f_6155_na_mask, 'Vitamin C supplementation'] = np.nan # process NA

# Multivitamin
merged_df['Multivitamin supplementation'] = merged_df[f_6155_cols].eq(7).any(axis=1).astype(int)
merged_df.loc[f_6155_na_mask, 'Multivitamin supplementation'] = np.nan # process NA

In [ ]:
## Glucosamine, iron, selenium, calcium (f 6179)

f_6179_cols = merged_df.columns[merged_df.columns.str.startswith('f.6179.0.')]
f_6179_na_mask = (merged_df['f.6179.0.0'].eq(-3)) | (merged_df['f.6179.0.0'].isna() == True)

# Glucosamine supplementation
merged_df['Glucosamine supplementation'] = merged_df[f_6179_cols].eq(2).any(axis=1).astype(int)
merged_df.loc[f_6179_na_mask, 'Glucosamine supplementation'] = np.nan # process NA

# Iron supplementation
merged_df['Iron supplementation'] = merged_df[f_6179_cols].eq(5).any(axis=1).astype(int)
merged_df.loc[f_6179_na_mask, 'Iron supplementation'] = np.nan # process NA

# Selenium supplementation
merged_df['Selenium supplementation'] = merged_df[f_6179_cols].eq(6).any(axis=1).astype(int)
merged_df.loc[f_6179_na_mask, 'Selenium supplementation'] = np.nan # process NA

# Calcium supplementation
merged_df['Calcium supplementation'] = merged_df[f_6179_cols].eq(3).any(axis=1).astype(int)
merged_df.loc[f_6179_na_mask, 'Calcium supplementation'] = np.nan # process NA

### Salt consumption

**Self-reported salt added to food**

In [ ]:
# Self-report: salt added to food
# Ordinal
# 0 = never/rarely, 1 = sometimes, 2 = usually, 3 = always

merged_df['Salt added to food'] = merged_df['f.1478.0.0'].replace({
    -3: np.nan, # prefer not to answer
    1: 0, # never/rarely
    2: 1, # sometimes
    3: 2, # usually
    4: 3, # always
})

**Urinary Na+ excretion**

- Sodium consumption estimated by urine sodium:creatinine ratio (UNa:Cr)
- See Stuart et al., 2024 (https://pubmed.ncbi.nlm.nih.gov/38723778/). Shown to correlate with 24-hour sodium intake by dietary recall
- "In a steady state, renal excretion of creatinine remains relatively constant, and the urinary creatinine concentration therefore provides a measure of the state of dilution or concentration of the urine. This approach is widely used to estimate 24-hour excretion of sodium and other analytes, such as albumin and catecholamines, from spot urine samples."

In [ ]:
## UNa:Cr

# Use units mmol:mmol
# Converting creatinine to mmol -> divide by 1000
# Sodium in urine, f 30530: mmol
# Creatinine in urine, f 30510: micromole/L

merged_df['Urinary sodium-creatinine ratio'] = merged_df['f.30530.0.0'] / (merged_df['f.30510.0.0'] / 1000)

### BMI

In [ ]:
merged_df['Body mass index'] = merged_df['f.21001.0.0']

### Vitamin D

In [ ]:
merged_df['Plasma Vitamin D'] = merged_df['f.30890.0.0']

### Exercise

Final variable of interest: `summed_MET_mins_weekly`

Following IPAQ guidelines: https://biobank.ndph.ox.ac.uk/ukb/ukb/docs/ipaq_analysis.pdf

This brings the derived MET scores to the equivalent to what is found in the UKBB showcase (https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=54). The measures available in our datafile are outdated as of June 2023 (see https://biobank.ndph.ox.ac.uk/ukb/refer.cgi?id=2401)

Note: exclusions only removed/processed for summary variable, `summed_MET_mins_weekly`. Individual MET scores for walking/moderate/vigorous have not been processed for exclusions and should not be used.

In [ ]:
# Walking METs

# Truncate duration to 180 mins
walking_duration_truncated = merged_df['f.874.0.0'].clip(lower=None, upper=180, inplace=False)

# Set METs
merged_df['walking_MET_mins_weekly_custom'] = merged_df['f.864.0.0'] * walking_duration_truncated * 3.3

# Special conditions
walking_days_week_none = merged_df['f.864.0.0'].eq(0) # no walking
merged_df.loc[walking_days_week_none, 'walking_MET_mins_weekly_custom'] = 0 # otherwise NA, as f.874 is missing

walking_days_week_unable = merged_df['f.864.0.0'].eq(-2) # cannot walk - treat this as 0 days as per UKBB
merged_df.loc[walking_days_week_unable, 'walking_MET_mins_weekly_custom'] = 0 # treat unable as 0 days as per UKBB

walking_duration_low = merged_df['f.874.0.0'] < 10 
merged_df.loc[walking_duration_low, 'walking_MET_mins_weekly_custom'] = 0 # treat <10mins as none

In [ ]:
# Moderate activity METs

# Truncate duration to 180 mins
mod_duration_truncated = merged_df['f.894.0.0'].clip(lower=None, upper=180, inplace=False)

# Set METs
merged_df['moderate_activity_MET_mins_weekly_custom'] = merged_df['f.884.0.0'] * mod_duration_truncated * 4

# Special conditions
moderate_days_week_none = merged_df['f.884.0.0'].eq(0) # no moderate exercise
merged_df.loc[moderate_days_week_none, 'moderate_activity_MET_mins_weekly_custom'] = 0 # otherwise NA, as f.894 missing

moderate_duration_low = merged_df['f.894.0.0'] < 10
merged_df.loc[moderate_duration_low, 'moderate_activity_MET_mins_weekly_custom'] = 0 # treat <10mins as none

In [ ]:
# Vigorous activity METs

# Truncate duration to 180 mins
vig_duration_truncated = merged_df['f.914.0.0'].clip(lower=None, upper=180, inplace=False)

# Set
merged_df['vigorous_activity_MET_mins_weekly_custom'] = merged_df['f.904.0.0'] * vig_duration_truncated * 8

# Special conditions
vigorous_days_week_none = merged_df['f.904.0.0'].eq(0) # no vigorous exercise
merged_df.loc[vigorous_days_week_none, 'vigorous_activity_MET_mins_weekly_custom'] = 0 # otherwise NA, as f.914 missing

vigorous_duration_low = merged_df['f.914.0.0'] < 10
merged_df.loc[vigorous_duration_low, 'vigorous_activity_MET_mins_weekly_custom'] = 0 # treat <10mins as none

In [ ]:
# Exclusions

# Declined walking, days or duration
walking_days_week_decline = merged_df['f.864.0.0'].isin([-1, -3])
walking_duration_decline = merged_df['f.874.0.0'].isin([-1, -3])

# Declined moderate activity, days or duration
moderate_days_week_decline = merged_df['f.884.0.0'].isin([-1, -3])
moderate_duration_decline = merged_df['f.894.0.0'].isin([-1, -3])

# Declined vigorous activity, days or duration
vigorous_days_week_decline = merged_df['f.904.0.0'].isin([-1, -3])
vigorous_duration_decline = merged_df['f.914.0.0'].isin([-1, -3])

# Summarise declines
activity_decline_exclusion = (
    walking_days_week_decline | walking_duration_decline
    | moderate_days_week_decline | moderate_duration_decline
    | vigorous_days_week_decline | vigorous_duration_decline
)

# Above maximum value of 960 minutes / 16hr across all exercise, per day
total_duration_exclusion = merged_df[['f.874.0.0', 'f.894.0.0', 'f.914.0.0']].sum(axis=1) > 960

In [ ]:
# Sum METs

merged_df['Exercise (summed MET minutes per week)'] = merged_df[['walking_MET_mins_weekly_custom', 'moderate_activity_MET_mins_weekly_custom', 'vigorous_activity_MET_mins_weekly_custom']].sum(skipna=False, axis=1)

# Remove exclusions
merged_df.loc[activity_decline_exclusion, 'Exercise (summed MET minutes per week)'] = np.nan
merged_df.loc[total_duration_exclusion, 'Exercise (summed MET minutes per week)'] = np.nan

**Strenuous sports**

In [ ]:
# Performs strenous sports

f_6164_cols = merged_df.columns[merged_df.columns.str.startswith('f.6164.0')]
f_6164_na = (merged_df['f.6164.0.0'].isna() == True) | (merged_df['f.6164.0.0'].eq(-3))

merged_df['Strenuous sports'] = merged_df[f_6164_cols].eq(3).any(axis=1).astype(int)
merged_df.loc[f_6164_na, 'Strenuous sports'] = np.nan # process NA

In [ ]:
# Duration of strenous sports (ordinal)
# 0 if not performed

# Default coding:
# 1	Less than 15 minutes
# 2	Between 15 and 30 minutes
# 3	Between 30 minutes and 1 hour
# 4	Between 1 and 1.5 hours
# 5	Between 1.5 and 2 hours
# 6	Between 2 and 3 hours
# 7	Over 3 hours
# -1	Do not know
# -3	Prefer not to answer

no_strenuous_sports = merged_df['Strenuous sports'] == 0

f_1001_na = merged_df['f.1001.0.0'].isin([-1, -3])
merged_df['Duration of strenous sports'] = merged_df['f.1001.0.0']
merged_df.loc[no_strenuous_sports, 'Duration of strenous sports'] = 0
merged_df.loc[f_1001_na, 'Duration of strenous sports'] = np.nan

### Coffee & tea

In [ ]:
### Caffeinated coffee consumption
# `caffeinated_coffee_drinker`: binary variable

# Field 1498 (coffee cups/d) is unavailable / not downloaded
# Therefore, use 1508 (coffee type) instead
# If 1508 is missing, assume because they did not drink coffee in response to 1498 (as per UKB showcase notes for https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=1508)

coffee_type_missing_mask = merged_df['f.1508.0.0'].isna() == True
coffee_type_na_mask = merged_df['f.1508.0.0'].isin([-1, -3]) # decline to answer

merged_df['Caffeinated coffee drinker'] = merged_df['f.1508.0.0'].isin([2, 3, 4]).astype(int) # instant, ground, other (i.e., not decaf)
merged_df.loc[coffee_type_missing_mask, 'Caffeinated coffee drinker'] = 0  # assume do not drink coffee
merged_df.loc[coffee_type_na_mask, 'Caffeinated coffee drinker'] = np.nan

In [ ]:
### Tea consumption
# `tea_intake`: numerical

# Special value of -10 for field 1488: "Less than one" -> code as 0.5 cusp/day
# As zero was also an option

tea_intake_na_mask = merged_df['f.1488.0.0'].isin([-1, -3]) # decline to answer
tea_intake_below_one_mask = merged_df['f.1488.0.0'].eq(-10)

merged_df['Tea intake'] = merged_df['f.1488.0.0']
merged_df.loc[tea_intake_below_one_mask, 'Tea intake'] = 0.5
merged_df.loc[tea_intake_na_mask, 'Tea intake'] = np.nan

### Alcohol

In [ ]:
merged_df['Alcohol intake'] = merged_df['f.1558.0.0'].replace({
    -3: np.nan, # decline to answer
    6: 0, # never
    5: 1, # special occasions
    4: 2, # 1-3 times a month
    3: 3, # 1-2 times a week
    2: 4, # 3-4 times a week
    1: 5, # daily or almost daily
})

### Smoking

In [ ]:
# Current tobacco smoking
# Ordinal variable
# 0 = no, 1 = occasionally, 2 = most/all days

merged_df['Current smoking frequency'] = merged_df['f.1239.0.0'].replace({
    -3: np.nan, # decline to answer
    0: 0, # no
    2: 1, # occasionally
    1: 2, # most days
})

In [ ]:
# Past tobacco smoking
# Ordinal variable
# 0 = no / tried once or twoce, 1 = occasionally, 2 = most or all days

merged_df['Past smoking frequency'] = merged_df['f.1249.0.0'].replace({
    -3: np.nan, # decline to answer
    4: 0, # never
    3: 0, # tried once or twice
    2: 1, # occasionally
    1: 2, # most days
})

### Sleep

In [ ]:
# Sleep duration
# Numerical

# Process NA values
merged_df['sleep_duration_hours'] = merged_df['f.1160.0.0'].replace({
    -1: np.nan, # do not know
    -3: np.nan, # prefer not to answer
})

# One-hot encode as categorical variable
# Normal sleep duration: ≥7 to <9 h/day (Sun et al., 2022 https://pubmed.ncbi.nlm.nih.gov/36319053/)
merged_df['Normal sleep duration'] = ((merged_df['sleep_duration_hours'] >= 7) & (merged_df['sleep_duration_hours'] < 9)).astype(int)

# Process NA
merged_df.loc[merged_df['sleep_duration_hours'].isna() == True, 'Normal sleep duration'] = np.nan

In [ ]:
# Insomnia
# Ordinal variable
# 0 = never/rarely, 1 = sometimes, 2 = usually

merged_df['Insomnia frequency'] = merged_df['f.1200.0.0'].replace({
    -3: np.nan, # decline to answer
    1: 0, # never/rarely
    2: 1, # sometimes
    3: 2, # usually
})

In [ ]:
# Snoring
# Binary variable

# ! Large number of do not know / prefer not to answer
# Possibly due to wording: "Does your partner or a close relative or friend complain about your snoring?"

merged_df['Snoring'] = merged_df['f.1210.0.0'].replace({
    -1: np.nan, # do not know
    -3: np.nan, # prefer not to answer
    2: 0, # no
    1: 1, # yes
})

In [ ]:
# Daytime sleeping
# Ordinal variable
# 0 = never/rarely, 1 = sometimes, 2 = often / all the time

# ! "All the time" was removed during study period, so small n., group together with often

merged_df['Daytime sleeping frequency'] = merged_df['f.1220.0.0'].replace({
    -1: np.nan, # do not know
    -3: np.nan, # prefer not to answer
    0: 0, # never/rarely
    1: 1, # sometimes
    2: 2, # often
    3: 2, # all the time
})

# Load glaucoma, control, & exclusion variables

In [ ]:
glaucoma_prevalence_def_df = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/emma_summer2023/william_glaucoma_def/glaucoma_defs_treated_diagnosed.txt').rename(columns={'Participant_ID':'f.eid'})

In [ ]:
exclusion_TUD = glaucoma_prevalence_def_df['glaucoma_final_def_limitedexclusions'] == 'Treated_Undiagnosed'
exclusion_other = glaucoma_prevalence_def_df['glaucoma_final_def_limitedexclusions'].isna() == True
exclusion_any = exclusion_TUD | exclusion_other

In [ ]:
glaucoma_prevalence_def_df['Exclusion'] = 0
glaucoma_prevalence_def_df.loc[exclusion_any, 'Exclusion'] = 1

In [ ]:
glaucoma_prevalence_def_df['Glaucoma (prevalent D|TD)'] = glaucoma_prevalence_def_df['glaucoma_final_def_limitedexclusions'].replace({
    'Control': 'Control',
    'Untreated_Diagnosed': 'Glaucoma',
    'Treated_Diagnosed': 'Glaucoma',
    'Treated_Undiagnosed': np.nan,
})
glaucoma_prevalence_def_df.loc[exclusion_any, 'Glaucoma (prevalent D|TD)'] = np.nan

In [ ]:
# Merge

merged_df = pd.merge(merged_df, glaucoma_prevalence_def_df[['f.eid', 'Exclusion', 'Glaucoma (prevalent D|TD)']], on='f.eid', how='outer')

In [ ]:
# N for flowchart

In [ ]:
# Merge

df_for_n = pd.merge(merged_df[['f.eid', 'IOP_available']], glaucoma_prevalence_def_df, on='f.eid', how='outer')

In [ ]:
df_for_n_IOP = df_for_n[df_for_n['IOP_available'] == 1]

In [ ]:
exclusion_TUD_df_for_n = df_for_n_IOP['glaucoma_final_def_limitedexclusions'] == 'Treated_Undiagnosed'
exclusion_other_df_for_n = df_for_n_IOP['glaucoma_final_def_limitedexclusions'].isna() == True
exclusion_any_df_for_n = exclusion_TUD_df_for_n | exclusion_other_df_for_n

In [ ]:
exclusion_any_df_for_n.sum()

In [ ]:
exclusion_TUD_df_for_n.sum()

In [ ]:
exclusion_other_df_for_n.sum()

In [ ]:
exclusion_TUD_df_for_n.

In [ ]:
(df_for_n_IOP['glaucoma_final_def_limitedexclusions'] == 'Treated_Undiagnosed').value_counts()

In [ ]:
(df_for_n_IOP['glaucoma_final_def_limitedexclusions'] == 'Treated_Undiagnosed').value_counts()

In [ ]:
df_for_n

# Subcohorting

In [ ]:
# IOP available subcohort = IOP data available & not excluded as a control

IOP_available_mask = merged_df['IOP_available'] == 1
not_excluded_mask = merged_df['Exclusion'] == 0

IOP_subcohort_mask = IOP_available_mask & not_excluded_mask

merged_df['IOP subcohort'] = 0
merged_df.loc[IOP_subcohort_mask, 'IOP subcohort'] = 1

# Training/test splits

In [ ]:
def stratify_split(df, mask_dict, test_proportion, split_col_name):
    for name, mask in mask_dict.items():
        subgroup_df = df[mask]
        subgroup_test = subgroup_df.sample(frac=test_proportion, replace=False, random_state=2024)
        df.loc[subgroup_test.index.values, split_col_name] = 'test'
        df.loc[subgroup_df.index.difference(subgroup_test.index.values), split_col_name] = 'train'

In [ ]:
def qc_split(df, mask_dict, split_col_name):
    test_proportion = (df[split_col_name] == 'test').sum() / len(df)
    print(f'Split col NA: {df[split_col_name].isna().sum()}')
    print(f'Test set proportion: {test_proportion}')
    print(f'Split col value counts: {df[split_col_name].value_counts()}')
    for name, mask in mask_dict.items():
        test_proportion = (df[mask][split_col_name] == 'test').sum() / len(df[mask])
        print(f'\n {name}')
        print(f'Test set proportion: {test_proportion}')
        print(df[mask][split_col_name].value_counts())

In [ ]:
### 90/10 split

split_col = 'training_test_split_90_10'
merged_df[split_col] = np.nan

glaucoma_mask = merged_df['Glaucoma (prevalent D|TD)'] == 'Glaucoma'
control_mask = merged_df['Glaucoma (prevalent D|TD)'] == 'Control'

# IOP subcohort
mask_dict = {
    'Glaucoma': IOP_subcohort_mask & glaucoma_mask,
    'Control': IOP_subcohort_mask & control_mask,
}
stratify_split(
    df=merged_df,
    mask_dict=mask_dict,
    test_proportion=0.1,
    split_col_name=split_col
)
qc_split(merged_df, mask_dict, split_col)

# Remaining study
mask_dict = {
    'Glaucoma': ~IOP_subcohort_mask & glaucoma_mask,
    'Control': ~IOP_subcohort_mask & control_mask,
}
stratify_split(
    df=merged_df,
    mask_dict=mask_dict,
    test_proportion=0.1,
    split_col_name=split_col
)
qc_split(merged_df, mask_dict, split_col)

# Save final derived data

In [ ]:
# Save all columns

merged_df.to_pickle('./data/derived/mixed_derived_and_extracted_merged.pkl')

In [ ]:
# Select only columns we have created / manipulated
# Note: will still include some redundant variables, that won't be used as features

cols_to_drop = np.concatenate((ukb49508_cols_to_use, ukb675501_cols_to_use, ukb51745_cols_to_use))
cols_to_drop = np.delete(cols_to_drop, np.where(cols_to_drop=='f.eid'))
df_to_save = merged_df.drop(cols_to_drop, axis=1, inplace=False)

In [ ]:
df_to_save.to_pickle('./data/derived/derived_cols_merged.pkl')

In [ ]:
df_to_save.to_csv('./data/derived/derived_cols_merged.tsv', sep='\t', index=False)